In [3]:
# Common
import os
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import clear_output as cls

# Data 
from tensorflow import io
import tensorflow.data as tfd
import tensorflow.image as tfi
from keras.preprocessing.image import ImageDataGenerator

# Data Visualization
import plotly.express as px
import matplotlib.pyplot as plt

# Model
import tensorflow_hub as hub
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers.schedules.learning_rate_schedule import PiecewiseConstantDecay

# Model Compiling
from keras.metrics import SparseCategoricalAccuracy, SparseTopKCategoricalAccuracy

# Callbacks 
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
PATCH_SIZE = 4    # 2 x 2
IMAGE_SIZE = 256  # The sizes reduced because of memory constraints.
N_CHANNELS = 3
BATCH_SIZE = 128  # Depending on the resource
AUTO = tfd.AUTOTUNE 
EXP_FACTOR = 2    # Expanding Factor

In [5]:
def decode_image(image):
    
    image = tfi.decode_jpeg(image, channels=3)
    # Here you can apply augmentation
    image = tfi.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = tf.cast(image, tf.float32)/255.
    
    return image

def ProcessData(example):

    truth = {
        "label" : io.FixedLenFeature([], tf.int64),
        "image" : io.FixedLenFeature([], tf.string),
    }
    
    example = io.parse_single_example(example, truth)
    
    image, label = example['image'], example['label']
    
    image = decode_image(image)
    
    return image, label

In [6]:
def show_image(image, title=None, show=False):
    plt.imshow(image)
    plt.axis('off')
    plt.title(title)
    if show:
        plt.show()

def show_images(data, class_names, model=None, GRID=[5,6], SIZE=(30,25)):

    # Configuration
    n_rows, n_cols = GRID
    n_images = n_rows * n_cols
    plt.figure(figsize=SIZE)

    # Iterate through the Data
    i = 1
    for images, labels in iter(data):
        
        # Select Image and label randomly
        id = np.random.randint(len(images))
        image, label = tf.expand_dims(images[id], axis=0), class_names[int(labels[id])]

        # Make prediction
        if model is not None:
            prediction = model.predict(image)[0]
            score = np.round(max(prediction), 2)
            pred = class_names[np.argmax(prediction)]
            title = f"True : {label}\nPred : {pred}\nScore : {score}"
        else:
            title = f"{label}"
        
        # Plot Image
        plt.subplot(n_rows, n_cols, i)
        show_image(image[0], title=title, show=False)

        # Clear Unnecessary text
        cls()

        # Once done break loop
        i+=1
        if i>n_images:
            break
    
    # Show Final Plot
    plt.show()

In [7]:
# Specify Main Paths
train_path = './Animal-Data-V2/Data-V2/Training Data'
tfr_train_path = './Animal-Data-V2/Data-V2/TFRecords/Training'

valid_path = './Animal-Data-V2/Data-V2/Validation Data'
tfr_valid_path = './Animal-Data-V2/Data-V2/TFRecords/Validation'

test_path = './Animal-Data-V2/Data-V2/Testing Data'
tfr_test_path = './Animal-Data-V2/Data-V2/TFRecords/Testing'